## Overview
The schema for this dataset is shown. As discussed in the steps for data preparation there is a common pattern to preparing operational business data for analysis. There is always a timestamp that captures when the data was captured. Because the objective of this analysis is to segment revenue by various factors, we will define these factors and the attributes for each factor. The factors are customers, products and geography. The sequence of processing steps in this notebook follows the description of the key steps in the concepts document.

## Schema for the Olist Dataset

![](../images/olist_schema.png)

In [1]:
import pandas as pd
fp_orders = "../data/olist_raw/olist_orders_dataset.csv"
fp_order_items = "../data/olist_raw/olist_order_items_dataset.csv"
df_orders = pd.read_csv(fp_orders)
df_order_items = pd.read_csv(fp_order_items)


## Filter and Aggregate Steps for Customer Data

In [2]:
cols = df_orders.columns.tolist() + df_order_items.columns.tolist()

In [3]:
cols

['order_id',
 'customer_id',
 'order_status',
 'order_purchase_timestamp',
 'order_approved_at',
 'order_delivered_carrier_date',
 'order_delivered_customer_date',
 'order_estimated_delivery_date',
 'order_id',
 'order_item_id',
 'product_id',
 'seller_id',
 'shipping_limit_date',
 'price',
 'freight_value']

In [4]:
cols_needed = ["order_id",  'customer_id', "order_purchase_timestamp", "order_item_id", "product_id", "price"]

In [5]:
df_daily_ord_raw = pd.merge(df_orders, df_order_items, on = "order_id")

In [6]:
df_daily_ord_raw = df_daily_ord_raw[cols_needed]

In [7]:
df_daily_ord_raw["order_purchase_timestamp"] = pd.to_datetime(df_daily_ord_raw["order_purchase_timestamp"])

In [8]:
df_daily_ord_raw

,order_id,customer_id,order_purchase_timestamp,order_item_id,product_id,price
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,1,87285b34884572647811a353c7ac498a,29.99
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,2018-07-24 20:41:37,1,595fac2a385ac33a80bd5114aec74eb8,118.70
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,2018-08-08 08:38:49,1,aa4383b373c6aca5d8797843e5594415,159.90
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,2017-11-18 19:28:06,1,d0b61bfb1de832b15ba9d266ca96e5b0,45.00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,2018-02-13 21:18:39,1,65266b2da20d04dbe00c5c2d3bb7859e,19.90
...,...,...,...,...,...,...
112645,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,2018-02-06 12:58:58,1,f1d4ce8c6dd66c47bbaa8c6781c2a923,174.90
112646,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,2017-08-27 14:46:43,1,b80910977a37536adeddd63663f916ad,205.99
112647,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,2018-01-08 21:28:27,1,d1c427060a0f73f6b889a5c7c61f2ac4,179.99
112648,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,2018-01-08 21:28:27,2,d1c427060a0f73f6b889a5c7c61f2ac4,179.99


In [9]:
fp_customer = "../data/olist_raw/olist_customers_dataset.csv"
df_cust = pd.read_csv(fp_customer)
cust_cols = ["customer_id","customer_zip_code_prefix", "customer_city", "customer_state"]
df_cust = df_cust[cust_cols]

In [10]:
df_daily_ord_raw =  pd.merge(df_daily_ord_raw, df_cust, on = "customer_id")

In [11]:
df_daily_ord_raw = df_daily_ord_raw.sort_values(by="order_id").reset_index()
fp_daily_ord_raw = "../data/olist_prepared/olist_daily_orders_prepared.csv"
df_daily_ord_raw.to_csv(fp_daily_ord_raw, index=False)

In [12]:
ord_qty_vc = df_daily_ord_raw.order_id.value_counts().values

In [13]:
ord_qty_vc

array([21, 20, 20, ...,  1,  1,  1], shape=(98666,))

In [14]:
df_gb = df_daily_ord_raw.groupby([df_daily_ord_raw.order_purchase_timestamp.dt.year,\
                                  df_daily_ord_raw.order_purchase_timestamp.dt.day_of_year])

In [15]:
df_prod_purchases = df_daily_ord_raw.product_id.value_counts().reset_index()

In [16]:
len(df_daily_ord_raw.product_id.unique())

32951

In [17]:
from matplotlib import pyplot as plt 

In [18]:
df_prod_purchases["count"].quantile(q=0.8)

np.float64(3.0)

## Filter and Aggregate Steps for Product Data

In [19]:
df_prod_purchases = df_prod_purchases.sort_values(by="count", ascending=False)
fp_prod_purch = "../data/olist_prepared/olist_2_year_prod_purchases.csv"
df_prod_purchases["rank"] = df_prod_purchases.index + 1
df_prod_purchases.to_csv(fp_prod_purch, index=False)

In [20]:
df_prod_purchases.head(20)

,product_id,count,rank
0,aca2eb7d00ea1a7b8ebd4e68314663af,527,1
1,99a4788cb24856965c36a24e339b6058,488,2
2,422879e10f46682990de24d770e7f83d,484,3
3,389d119b48cf3043d311335e499d9c6b,392,4
4,368c6c730842d78016ad823897a372db,388,5
5,53759a2ecddad2bb87a079a1f1519f73,373,6
6,d1c427060a0f73f6b889a5c7c61f2ac4,343,7
7,53b36df67ebb7c41585e8d54d6772e08,323,8
8,154e7e31ebfa092203795c972e5804a6,281,9
9,3dd2a17168ec895c781a9191c1e95ad7,274,10


In [21]:
df_prod_purchases["count"].unique().shape

(138,)

In [22]:
df_cust_order_count = df_daily_ord_raw.groupby(["customer_id"])["order_id"].count().reset_index()

In [23]:
df_cust_order_count.columns = ["customer_id", "order_count"]

In [24]:
df_cust_order_count = df_cust_order_count.sort_values(by="order_count", ascending=False).reset_index()